# 03 - Validación cruzada y GridSearchCV

En este notebook aprenderemos a:

- Automatizar la búsqueda del mejor modelo
- Usar validación cruzada
- Seleccionar hiperparámetros óptimos

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

df = pd.read_csv(
    "../data/SMSSpamCollection",
    sep="\t",
    header=None,
    names=["label", "text"]
)

df["label"] = df["label"].map({"ham": 0, "spam": 1})

X = df["text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

## ¿Qué es validación cruzada?

En lugar de entrenar una sola vez, dividimos el conjunto de entrenamiento en varias partes (folds).

Ejemplo con 5 folds:
- Se entrena 5 veces
- Cada vez usando una parte diferente como validación
- El resultado final es el promedio

Esto reduce el riesgo de depender de una sola división de datos.

In [ ]:
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", LogisticRegression(max_iter=400))
])

## ¿Qué son hiperparámetros?

Son configuraciones que nosotros elegimos antes de entrenar el modelo.

Ejemplos:
- ngram_range
- min_df
- C (regularización)
- class_weight

GridSearch probará todas las combinaciones posibles.

In [ ]:
param_grid = {
    "tfidf__ngram_range": [(1,1), (1,2)],
    "tfidf__min_df": [1, 2, 5],
    "tfidf__max_df": [0.9, 0.95, 1.0],
    "clf__C": [0.5, 1.0, 2.0],
    "clf__class_weight": [None, "balanced"]
}

grid = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    scoring="f1",   # métrica principal
    cv=5,           # 5 folds
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

GridSearch hará:

- 2 valores de ngram
- 3 de min_df
- 3 de max_df
- 3 de C
- 2 de class_weight

Total combinaciones:
2 × 3 × 3 × 3 × 2 = 108 modelos

Y cada uno con 5 folds:
108 × 5 = 540 entrenamientos

In [ ]:
print("Mejores parámetros:")
print(grid.best_params_)

print("\nMejor F1 (CV):")
print(grid.best_score_)

# Evaluación en el conjunto de test
best_model = grid.best_estimator_

y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=["ham","spam"]))

# Optimización Recall

In [ ]:
grid_recall = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    scoring="recall",
    cv=5,
    n_jobs=-1
)

grid_recall.fit(X_train, y_train)

print("Mejores parámetros (optimizado para recall):")
print(grid_recall.best_params_)